In [2]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from google.colab import files

from sklearn.model_selection import RandomizedSearchCV, KFold, cross_validate

import scipy.stats as stats

In [ ]:
uploaded = files.upload()

In [ ]:
dataset_path = "/content/housing_price_dataset.csv"

dataframe = pd.read_csv(dataset_path)

#Remocão de valores negativos no preco
dataframe = dataframe[dataframe['Price'] >=0]

print(dataframe.head())

In [ ]:
# CÉLULA 3: (NOVO) ENGENHARIA DE FEATURES
# Criamos novas features a partir das existentes para tentar melhorar o modelo.
# Esta etapa é feita ANTES de separar X e y.
print("\n--- Aplicando Engenharia de Features ---")

# 1. Criando a feature 'HouseAge' (Idade do Imóvel)
current_year = 2025 # Usando o ano de referência da nossa conversa
dataframe['HouseAge'] = current_year - dataframe['YearBuilt']

# 2. Criando a feature 'BathToBedRatio' (Proporção de Banheiros por Quarto)
dataframe['BathToBedRatio'] = dataframe['Bathrooms'] / (dataframe['Bedrooms'] + 1)

# 3. Criando um novo DataFrame e removendo a coluna original 'YearBuilt'
dataframe_eng = dataframe.drop(columns=['YearBuilt'])

print("\n--- DataFrame Após Engenharia de Features (Primeiras 5 Linhas) ---")
print(dataframe_eng.head())

In [ ]:
# ANÁLISE DOS DADOS

# Vemos que não temos valores nulos no dataset, e que temos uma coluna categórica (Neighborhood)
# com valores do tipo object, então terá que ser tratado na fase de pré processamento
dataframe_eng.info()

In [ ]:
dataframe_eng.describe()

In [ ]:
# PRÉ-PROCESSAMENTO

# Converter a coluna categórica para valores númericos
# Escalonar as colunas

# Separando features
y = dataframe_eng['Price']

X = dataframe_eng.drop(columns=['Price'])

print('Variáveis independentes (Features)')
print(X.head())

print('Varíavel dependente (Alvo)')
print(y.head())

In [ ]:
# One-Hot Enconding para converter os valores de Neighborhood
# StandardScaler para escalonar as features
# Vamos fazer isso usando o ColumnTransformer, para criar um "pré-processador"

# Identificando as colunas
categorical_features = ['Neighborhood']
numerical_features = X.select_dtypes(include = np.number).columns.tolist()

print(f'Colunas categóricas: {categorical_features}')
print(f'Colunas para escalar: {numerical_features}')

# Criando o pré-processador
preprocessor = ColumnTransformer(
    transformers=[
      ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
      ('poly_num', Pipeline(steps = [('poly', PolynomialFeatures(degree = 2, include_bias = False)), ('scaler', StandardScaler())]), numerical_features),
    ],
    remainder = 'passthrough'
)

In [ ]:
# DIVISÃO DE DADOS EM DADOS DE TREINAMENTO E TESTE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Tamanho dos conjuntos de dados')
print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_test: {y_test.shape}')

In [ ]:
# TREINAMENTO DOS MODELOS COM BUSCA POR HIPERPARÂMETROS E COM AS
# MÉTRICAS DA VALIDAÇÃO CRUZADA (DEMORADO EM)

models = {
    'Regressão Linear': LinearRegression(),
    'Lasso': Lasso(random_state=42, max_iter = 2000),
    'SVR': SVR(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

param_grids = {
    'Lasso': {'regressor__alpha': np.logspace(-4, 1, 100)},
    'SVR': {'regressor__kernel': ['rbf', 'linear'], 'regressor__C': [0.1, 1, 10, 100], 'regressor__gamma': ['scale', 'auto'], 'regressor__epsilon': [0.01, 0.1, 0.2]},
    'Random Forest': {'regressor__n_estimators': [100, 200, 300], 'regressor__max_depth': [10, 20, 30, None], 'regressor__min_samples_leaf': [1, 2, 4]},
    'Gradient Boosting': {'regressor__n_estimators': [100, 200, 300], 'regressor__learning_rate': [0.01, 0.05, 0.1], 'regressor__max_depth': [3, 5, 7]}
}

final_results = {}
best_estimators = {}

for name, model in models.items():
    print(f"\nPROCESSANDO MODELO: {name}")

    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])

    # Busca de Hiperparâmetros
    if name in param_grids:
        print("\nBuscando hiperparâmetros...")
        random_search = RandomizedSearchCV(pipeline, param_distributions=param_grids[name], n_iter=20, cv=5, verbose=1, random_state=42, n_jobs=-1, scoring='r2')
        random_search.fit(X_train, y_train)
        best_model = random_search.best_estimator_
        print(f"Melhores parâmetros para {name}: {random_search.best_params_}")
    else:
        best_model = pipeline

    best_estimators[name] = best_model

    # Avaliação
    print("\nIniciando avaliação robusta")

    list_of_r2s, list_of_maes, list_of_rmses = [], [], []
    n_repeats = 30

    scoring_metrics = ['r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error']

    for i in range(n_repeats):
        cv_strategy = KFold(n_splits=5, shuffle=True, random_state=i)

        scores_dict = cross_validate(best_model, X_train, y_train, cv=cv_strategy, scoring=scoring_metrics, n_jobs=-1)

        list_of_r2s.append(scores_dict['test_r2'].mean())
        list_of_maes.append(-scores_dict['test_neg_mean_absolute_error'].mean())
        list_of_rmses.append(-scores_dict['test_neg_root_mean_squared_error'].mean())

    final_results[name] = {
        'Média R²': np.mean(list_of_r2s),
        'Std R²': np.std(list_of_r2s),
        'Média MAE': np.mean(list_of_maes),
        'Std MAE': np.std(list_of_maes),
        'Média RMSE': np.mean(list_of_rmses),
        'Std RMSE': np.std(list_of_rmses)
    }

    print(f"Resultado final para {name}: R² = {final_results[name]['Média R²']:.4f}, MAE = {final_results[name]['Média MAE']:,.2f}, RMSE = {final_results[name]['Média RMSE']:,.2f}")


# RESULTADOS

print(f"\nTABELA COMPARATIVA FINAL")
df_final_results = pd.DataFrame(final_results).T
df_final_results = df_final_results.sort_values(by='Média R²', ascending=False)
print(df_final_results)

plt.figure(figsize=(12, 7))
sns.barplot(x=df_final_results.index, y=df_final_results['Média R²'])
plt.errorbar(x=df_final_results.index, y=df_final_results['Média R²'], yerr=df_final_results['Std R²'], fmt='none', c='black', capsize=5)
plt.title('Comparação Robusta do R² entre os Modelos (Média de 30x5-Fold CV)', fontsize=16)
plt.xlabel('Modelo Otimizado', fontsize=12)
plt.ylabel('R² (Coeficiente de Determinação)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, max(df_final_results['Média R²']) * 1.1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

print(f"\nANÁLISE GRÁFICA DO MELHOR MODELO")

best_model_name = df_final_results.index[0]
print(f"O melhor modelo, com base no R² médio, foi: '{best_model_name}'")

final_model = best_estimators[best_model_name]

# Retreinando o melhor modelo com todos os dados de treino para a previsão final
final_model.fit(X_train, y_train)

y_pred_final = final_model.predict(X_test)

# Gráfico de Previsões vs. Valores Reais
plt.figure(figsize=(10, 8))
sns.scatterplot(x=y_test, y=y_pred_final, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.title(f'Previsões vs. Valores Reais - {best_model_name}', fontsize=16)
plt.xlabel('Preços Reais (y_test)', fontsize=12)
plt.ylabel('Preços Previstos (y_pred)', fontsize=12)
plt.grid(True)
plt.show()

# Histograma dos Resíduos
residuals = y_test - y_pred_final

plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=50)
plt.axvline(x=0, color='red', linestyle='--')
plt.title(f'Distribuição dos Resíduos - {best_model_name}', fontsize=16)
plt.xlabel('Resíduo (Preço Real - Preço Previsto)', fontsize=12)
plt.ylabel('Frequência', fontsize=12)
plt.show()

print("\nAnálise dos Resíduos:")
print(f"  Média dos resíduos: R$ {residuals.mean():,.2f}")